In [ ]:
# not better to run a simple hf hosted model?

In [21]:
from cot import Collection
worldtree = Collection(["worldtree"], verbose=False)
# worldtree.select(split="train", number_samples=1, random_samples=True, seed=0)

Loading worldtree...


In [22]:
worldtree

ValueError: 4 columns passed, passed data had 3 columns

In [12]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains.llm import LLMChain
import json
from langchain.chat_models import ChatOpenAI

"""CoT Chain"""

llm = ChatOpenAI(temperature=.0,model_name="gpt-3.5-turbo") #ADA #for chat: gpt-3.5-turbo




template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

{cot_trigger}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger"], template=template)
cot_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="cot")
"""answer extraction chain"""

extraction_template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

Cot: {cot_trigger}{cot}
{answer_extraction}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction"], template=extraction_template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="predicted_answer")

"""CoT-Ans_extraction chain"""

from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[cot_chain, answer_chain],
                                input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction"],
                                output_variables=["cot", "predicted_answer"],
                                verbose=True)

In [16]:
input_dict = {
    "chain":overall_chain,
    "instruction": "Be faithful and a little hopeful",
    "cot_trigger": "Answer: Let's think step by step.",
    "answer_extraction": "Therefore, among A through D, the answer is",
    'model':"gpt-3.5-turbo",
    'temperature': 0,
    'max_tokens': 800 
}
worldtree.generate_extract_flexible(input_dict=input_dict,name='worldtree',split='train')

Generating worldtree...


> Entering new SequentialChain chain...

> Finished chain.


In [17]:
worldtree.evaluate(overwrite=True)

{'worldtree': {'train': {'accuracy': {'gpt-3.5-turbo': {"Be faithful and a little hopeful_Answer: Let's think step by step._Therefore, among A through D, the answer is": 1.0}}}}}